In [2]:
## required packages
import os, h5py, time
import numpy as np
from scipy import sparse

In [4]:
# file_dir = 'D:\\OpenSHAInterface\\'
file_dir = 'C:\\Users\Barry\\Desktop\\OpenSHAInterface\\'
im_dir = file_dir + 'im_output\\'
rup_meta_file = file_dir + 'rup_meta.hdf5'
sha_meta_file = file_dir + 'sha_meta.txt'
rup_num_start = 1
rup_num_end = 2

In [7]:
start_time = time.time()
im_in, rup_meta = read_OpenSHAInterface(im_dir,rup_meta_file,sha_meta_file,1,10)
print(time.time()-start_time)

0.11491012573242188


In [9]:
im_in.keys()

dict_keys(['247_0', '350_0', '1001_0', '1143_0', '1180_0', '1574_0', '2096_0', '2161_0', '2297_0'])

In [10]:
rup_meta.keys()

dict_keys(['247_0', '350_0', '1001_0', '1143_0', '1180_0', '1574_0', '2096_0', '2151_0', '2161_0', '2297_0'])

In [6]:
def read_OpenSHAInterface(im_dir, rup_meta_file, sha_meta_file, rup_num_start, rup_num_end):
	"""
	Read outputs from OpenSHAInterface wrapper
	
	Parameters
	----------
	im_dir : str
		directory of the **IM** outputs from OpenSHAInterface
	rup_meta_file : str
		full path for the file containing rupture metadata
	sha_meta_file : str
		full path for the file containing metadata from OpenSHAInterface
	rup_num_start : int
		starting rupture number (1 through nRups); default = 1
	rup_num_end : int
		ending rupture nubmer (1 through nRups); default = 1e10 (e.g., all ruptures)
		
	Returns
	-------
	im_in : float, dict
		dictionary containing the nonzero site indices, and **PGA** and **PGV** predictions (i.e., means, sigmas)
	rup_meta : float, dict
		dictionary containing the rupture scenario information (UCERF3 index, magnitude, mean annual rate)
	
	"""
	
	## open rup_meta_file and load in rupture information
	with h5py.File(rup_meta_file, 'r') as f:
		src = f.get('src')[:]
		rup = f.get('rup')[:]
		M = f.get('M')[:]
		rate = f.get('rate')[:]
	f.close()
		
	## number of rupture scenarios
	nRups = len(rup)
	
	## load sha_meta_file and extra information on nonzero IMs (sparse coordinates)
	sha_meta = np.loadtxt(sha_meta_file,delimiter=',',unpack=True)
	
	## extract start and end rupture numbers for importing 
	rup_start = sha_meta[1].astype(np.int32)
	rup_end = sha_meta[2].astype(np.int32)
	nFiles = len(rup_start)
	i_start = 0 if rup_num_start == 0 else next(ind-1 for ind in range(nFiles) if rup_start[ind] >= rup_num_start)
	i_end = nFiles-1 if rup_num_end >= rup_end[nFiles-1]+1 else next(ind for ind in range(nFiles) if rup_end[ind]+1 >= rup_num_end)
	
	## create list of ruptures to import based on input
	rup_list = np.arange(rup_num_start-1,min(rup_num_end,rup_end[nFiles-1]+1))
	
	## loop to impot im_data
	for i in range(i_start, i_end+1):
		## range of ruptures in file to be loaded
		rup_start_i = rup_start[i]
		rup_end_i = rup_end[i]
		
		## load file, store into im_data, then close
		im_file = 'im_' + str(rup_start_i) + '_' + str(rup_end_i) + '.hdf5' ## im file name to load
		with h5py.File(im_dir+im_file, 'r') as f:
			im_data = f.get('im')[:]
		f.close()
		
		## get the counting index for the rupture (not UCERF3 index)
		row = im_data[0].astype(np.int32)
		col = im_data[1].astype(np.int32) # load in nonzero cols now, easier to convert to integers
		
		## store ruptures that are in rup_list
		im_in = {}
		for j in np.unique(row):
			if j in rup_list:
				im_in.update({str(src[j])+'_'+str(rup[j]):{'site_nonzero':col[row==i], 
														'pga_mean':im_data[2][row==i], 
														'pga_sig_inter':im_data[3][row==i], 
														'pga_sig_intra':im_data[4][row==i], 
														'pgv_mean':im_data[5][row==i], 
														'pgv_sig_inter':im_data[6][row==i], 
														'pgv_sig_intra':im_data[7][row==i]}})
	
	## create rup_meta dictionary
	rup_meta = {}
	for i in rup_list:
		rup_meta.update({str(src[i])+'_'+str(rup[i]):{'M':np.round(M[i],decimals=2),
													'r':np.round(rate[i],decimals=6)}})

	##
	return im_in, rup_meta

In [ ]:
with h5py.File(file_dir+meta_rup_file, 'r') as f:
    src = f.get('src')[:]
    rup = f.get('rup')[:]
    M = f.get('M')[:]
    rate = f.get('rate')[:]
f.close()
nRups = len(rup)
nRups

In [ ]:
meta_rup = {}
for i in range(nRups):
    meta_rup.update({str(src[i])+'_'+str(rup[i]):{'M':np.round(M[i],decimals=2),
                                                  'r':np.round(rate[i],decimals=6)}})

In [ ]:
meta_sha = np.loadtxt(file_dir+meta_sha_file,delimiter=',',unpack=True)

In [ ]:
rup_start = meta_sha[1].astype(np.int32)
rup_end = meta_sha[2].astype(np.int32)
len_sparse = meta_sha[3].astype(np.int32)
nFiles = len(rup_start)
nFiles

In [ ]:
im_list = ['pga', 'pgv']

In [ ]:
i = 0
im_data = {}
im_file = 'im_' + str(rup_start[i]) + '_' + str(rup_end[i]) + '.hdf5'
with h5py.File(im_data_dir+im_file, 'r') as f:
    im_data = f.get('im')[:]
f.close()

In [ ]:
rup_end[i]-rup_start[i]

In [ ]:
row = im_data[0].astype(np.int32)
col = im_data[1].astype(np.int32)

In [ ]:
sparse_start = []
sparse_end = []
im_in = {}
start_time = time.time()
for i in np.unique(row):
    im_in.update({i:{'site':col[row==i], 
                     'pga':{'mean':im_data[2][row==i], 
                            'inter':im_data[3][row==i], 
                            'intra':im_data[4][row==i]}, 
                     'pgv':{'mean':im_data[5][row==i], 
                            'inter':im_data[6][row==i], 
                            'intra':im_data[7][row==i]}}})
print('time -------%10.4f' % (time.time() - start_time))

In [ ]:
next(j for j in range(len(row)) if row[j] > nRups*(i+1))

In [ ]:
pga_mean = sparse.coo_matrix((im_data[2], (row, col)))

In [ ]:
pga_mean

In [ ]:
np.min(im_data[2])

In [ ]:
np.unique(row)

In [ ]:
col[0:100]

In [ ]:
len(meta_sha[3])

In [ ]:
# for i in range(nRups):
#     im_data = {}
#     for im in im_list:
#         im_file = im + '_' + str(rup_start[i]) + '_' + str(rup_end[i]) + '.hdf5'
#         with h5py.File(im_data_dir+im_file, 'r') as f:
#             im_data.update({im: f.get(im)[:]})
#         f.close()

#     im_out = np.vstack([im_data['pga'],im_data['pgv'][2:5]])
#     save_name = 'im_' + str(rup_start[i]) + '_' + str(rup_end[i]) + '.hdf5'
#     with h5py.File(im_data_dir+save_name, 'w') as f:
#         f.create_dataset('im', data=im_out)
#     f.close()

In [ ]:
rup_num_start=2

In [ ]:
i_start = 0 if rup_num_start == 0 else next(ind-1 for ind in range(nFiles) if rup_start[ind] >= rup_num_start)
i_start

In [ ]:
rup_num_end = 99

In [ ]:
i_end = nFiles-1 if rup_num_end >= rup_end[nFiles-1]+1 else next(ind for ind in range(nFiles) if rup_end[ind]+1 >= rup_num_end)

In [ ]:
i_end

In [ ]:
a = range(rup_num_start-1,min(rup_num_end-1,rup_end[nFiles-1]))
a

In [ ]:
2 in np.array([3,2])

In [ ]:
im_data[2]

In [ ]:
for a in [1,2,3]:
    print(a)

In [ ]:
a = [0,0,0,1,1,1,2,2,2]
b = [0,1,2,0,1,2,0,1,2]
c = [2,3,5,0,2,3,0,0,1]

In [ ]:
d = sparse.coo_matrix((c,(a,b)))

In [ ]:
print(d.toarray())

In [ ]:
im_file = 'im_0_99.hdf5'
with h5py.File(im_dir+im_file, 'r') as f:
    im_data = f.get('im')[:]
f.close()
rows = im_data[0]
for i in np.unique(rows):
    print('i = %d: avg = %6.3f, std = %6.3f' % (i, np.mean(im_data[6][rows==i]), np.std(im_data[6][rows==i])))